In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  Lander_big.ts
tin_filename:  Lander_big
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "TopoRela_Forman_Tetra_Spark_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

spark_app_name: TopoRela_Forman_optimal_Lander_big_06192024_1852


### read input filesread input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 4959
root
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



### globally obtain VT relation

In [5]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [6]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### locally obtain VF, VE, EF, FT

In [7]:
from itertools import combinations
def get_Forman_rela_from_VT(Ver, VT):
    if VT:
        # store VF in multi_F, VE in multi_E, VV in pt_set
        multi_F = set()
        multi_E = set()
        # pt_set = set()
        
        FT = {}
        for tetra in VT:
            # VF
            # Generate combinations of 3 elements including Ver
            for comb in combinations(tetra, 3):
                if Ver in comb:
                    sorted_comb = sorted(comb, reverse=True)
                    multi_F.add(tuple(sorted_comb))
                    
            # VE
            for v in tetra:
                if v > Ver:
                    multi_E.add(tuple([v, Ver]))
                elif v < Ver:
                    multi_E.add(tuple([Ver, v]))
                # ignore if v == Ver
                
            # VV
            pt_set.update(tetra)
            
            # FT
            if Ver in tetra:
                index_ver = tetra.index(Ver)
                if index_ver == 0: # the first vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[2]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[1], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[0], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                        
                elif index_ver == 1: # the second vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[2]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[1], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[1], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                        
                elif index_ver == 2: # the second vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[2]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[2], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[1], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                        
                elif index_ver == 3: # the second vertex is the expected one
                    face_0 = [tetra[0], tetra[1], tetra[3]]
                    if tuple(face_0) in FT:
                        FT[tuple(face_0)].append(tetra)
                    else:
                        FT[tuple(face_0)] = [tetra]
                        
                    face_1 = [tetra[0], tetra[2], tetra[3]]
                    if tuple(face_1) in FT:
                        FT[tuple(face_1)].append(tetra)
                    else:
                        FT[tuple(face_1)] = [tetra]
                        
                    face_2 = [tetra[1], tetra[2], tetra[3]]
                    if tuple(face_2) in FT:
                        FT[tuple(face_2)].append(tetra)
                    else:
                        FT[tuple(face_2)] = [tetra]
                    
        # VF
        multi_F_list = sorted(multi_F) # save more time when using list(multi_F)
        
        # VE
        multi_E_list = sorted(multi_E)
        
        # VV
        # pt_set.remove(Ver)
        # pt_list = sorted(pt_set)
        
        multi_EF = {}
        for face in multi_F_list:
            # Generate combinations of 3 elements including Ver
            for edge in combinations(face, 2):
                if Ver in edge:
                    sorted_edge = sorted(edge, reverse=True)
                    if tuple(sorted_edge) in multi_EF:
                        multi_EF[tuple(sorted_edge)].append(face)
                    else:
                        multi_EF[tuple(sorted_edge)] = [face]
            
        return multi_F_list, multi_E_list, multi_EF, FT
            
# StructType for Forman relations
union_Forman_rela_schema = StructType([
    StructField("VF", ArrayType(ArrayType(IntegerType())),True), 
    StructField("VE", ArrayType(ArrayType(IntegerType())),True),
    StructField("EF", MapType(ArrayType(IntegerType()), ArrayType(ArrayType(IntegerType()))),True),
    StructField('FT', MapType(ArrayType(IntegerType()), ArrayType(ArrayType(IntegerType()))),True)
])

get_Forman_rela_from_VT_udf = udf(get_Forman_rela_from_VT, union_Forman_rela_schema)

df_Forman_rela = df_VT.withColumn("Forman_rela", get_Forman_rela_from_VT_udf(df_VT.Ver, df_VT.VT))
# df_VF = df_VF.select("Ver", "VF")
df_Forman_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- Forman_rela: struct (nullable = true)
 |    |-- VF: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- EF: map (nullable = true)
 |    |    |-- key: array
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- value: array (valueContainsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- FT: map (nullable = true)
 |    |    |-- key: array
 |    |    |    |-- element: integer (containsNull = true)
 |    |    |-- value: array (valueContainsNull = true)
 |    |    

In [8]:
t0 = time.time()

num_row = df_Forman_rela.count()

t1 = time.time()
t_count_Forman_rela = t1 - t0
print("Time cost:", t_count_Forman_rela)
# print("number of rows:", num_row)

Time cost: 8191.1910190582275


### compute the edges, faces, and tetrahedra within a lower star

In [9]:
def get_LS_EFT(Ver, VT, VF, VE):
    LS_VT = []
    LS_VF = []    
    LS_VE = []   
    for tetra in VT:
        if Ver >= max(tetra):
            LS_VT.append(tetra)
    for face in VF:
        if Ver >= max(face):
            LS_VF.append(face)
    for edge in VE:
        if Ver >= max(edge):
            LS_VE.append(edge)
            
    return LS_VT, LS_VF, LS_VE

get_LS_EFT_schema = StructType([
    StructField("LS_tetra", ArrayType(ArrayType(IntegerType())),True), 
    StructField('LS_face',ArrayType(ArrayType(IntegerType())),True),
    StructField('LS_edge',ArrayType(ArrayType(IntegerType())),True)
])

get_LS_EFT_udf = udf(get_LS_EFT, get_LS_EFT_schema)

In [10]:
df_LS_EFT_init = df_Forman_rela.withColumn("LS_EFT", get_LS_EFT_udf(df_Forman_rela.Ver, df_Forman_rela.VT, df_Forman_rela.Forman_rela.VF, df_Forman_rela.Forman_rela.VE))

df_LS_EFT = df_LS_EFT_init.select('Ver', col('LS_EFT.LS_tetra').alias('LS_VT'), col('LS_EFT.LS_face').alias('LS_VF'), col('LS_EFT.LS_edge').alias('LS_VE'))
df_LS_EFT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- LS_VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- LS_VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### compute Forman gradient

In [11]:
from collections import deque
def get_Forman(Ver, LS_tetra, LS_face, LS_edge):
    d = 3
    k = 0
    ST = deque() # ST stores k-simplices belonging to the lower star, the initial value is the vertex Ver
    ST.append(Ver)
    # LT = LS_face # LT stores the top simplices belonging to the lower star, here are faces
    crit_cell = [] # crit_cell will store critical simplices
    Forman_VE = [] # store VE pairs
    Forman_EF = [] # store EF pairs
    Forman_FT = [] # store FT pairs
    
    if (len(LS_edge)+len(LS_face)+len(LS_tetra)) == 0: # ver is a local minimum
        crit_cell.append([Ver])
        return crit_cell, Forman_VE, Forman_EF, Forman_FT
    
    def num_unpaired_edges(tri_inner, crit_cell_inner, Forman_vec_pair_VE_inner, Forman_vec_pair_EF_inner):
        # compute the number of unpaired edges in tri_inner
        edge0 = [tri_inner[0], tri_inner[1]]
        edge1 = [tri_inner[0], tri_inner[2]]
        
        num_unpaired_edge = 2
        paired_edge = [] # stores edges in crit_cell_inner, Forman_vec_pair_VE_inner, and Forman_vec_pair_EF_inner
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 2: # crit_inner_temp is a critical edge
                paired_edge.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_VE_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, face)
            paired_edge.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_EF_inner: # pair_vec is a vector, it can be (ver, edge) or (edge, face)
            paired_edge.append(pair_vec[0])
                
        potential_paired_edge = [edge0, edge1]
        if edge0 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            potential_paired_edge.remove(edge0)
            
        if edge1 in paired_edge:
            num_unpaired_edge = num_unpaired_edge - 1
            potential_paired_edge.remove(edge1)
            
        return num_unpaired_edge, potential_paired_edge
    
    def get_EF_pair(ST_inner, crit_cell_inner, Forman_VE_inner, Forman_EF_inner):
        # this function is to get the EF pairs, E derives from CR while F derives from ST
        for i in range(len(ST_inner)):
            face = ST_inner[i]
            num_unpaired_edge, potential_paired_edge = num_unpaired_edges(face, crit_cell_inner, Forman_VE_inner, Forman_EF_inner)
            if num_unpaired_edge == 1:
                return [potential_paired_edge[0], face] # return the EF pair
            
    def num_unpaired_faces(tetra_inner, crit_cell_inner, Forman_vec_pair_EF_inner, Forman_vec_pair_FT_inner):
        # compute the number of unpaired faces in tetra_inner
        face0 = [tetra_inner[0], tetra_inner[1], tetra_inner[2]]
        face1 = [tetra_inner[0], tetra_inner[1], tetra_inner[3]]
        face2 = [tetra_inner[0], tetra_inner[2], tetra_inner[3]]
        
        num_unpaired_face = 3
        paired_face = [] # stores edges in crit_cell_inner, Forman_vec_pair_VE_inner, and Forman_vec_pair_EF_inner
        for crit_inner_temp in crit_cell_inner:
            if len(crit_inner_temp) == 3: # crit_inner_temp is a critical face
                paired_face.append(crit_inner_temp)
                
        for pair_vec in Forman_vec_pair_EF_inner: # pair_vec is a vector, it can be (edge, face) or (face, tetra)
            paired_face.append(pair_vec[1])
        for pair_vec in Forman_vec_pair_FT_inner: # pair_vec is a vector, it can be (edge, face) or (face, tetra)
            paired_face.append(pair_vec[0])
                
        potential_paired_face = [face0, face1, face2]
        if face0 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face0)
            
        if face1 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face1)
            
        if face2 in paired_face:
            num_unpaired_face = num_unpaired_face - 1
            potential_paired_face.remove(face2)
            
        return num_unpaired_face, potential_paired_face       
            
    def get_FT_pair(ST_inner, crit_cell_inner, Forman_EF_inner, Forman_FT_inner):
        # this function is to get the FT pairs, F derives from CR while T derives from ST
        for i in range(len(ST_inner)):
            tetra = ST_inner[i]
            num_unpaired_face, potential_paired_face = num_unpaired_faces(tetra, crit_cell_inner, Forman_EF_inner, Forman_FT_inner)
            if num_unpaired_face == 1:
                return [potential_paired_face[0], tetra] # return the FT pair
                
        
    
    while k <= d: # d == 3 for a tetrahedral mesh
        CR = ST # CR stores candidate critical simplices
        k = k + 1        
        if k == 1: # edges in the lower star
            ST = deque()
            for e in LS_edge:
                ST.append(e) # assign LS_edge to ST
        elif k == 2: # faces in the lower star
            ST = deque()
            for f in LS_face:
                ST.append(f) # assign LS_face to ST
        elif k == 3: # tetrahedra in the lower star
            ST = deque()
            for t in LS_tetra:
                ST.append(t) # assign LS_tetra to ST
        
        while len(CR) > 0:
            if k == 1: # get VE pairs
                pair_VE_V, pair_VE_E = CR.popleft(), ST.popleft()
                Forman_VE.append([pair_VE_V, pair_VE_E])
            elif k == 2: # get EF pairs
                pair_EF = get_EF_pair(ST, crit_cell, Forman_VE, Forman_EF)
                if pair_EF: # the EF pair exists
                    pair_EF_E = pair_EF[0]
                    pair_EF_F = pair_EF[1]
                    Forman_EF.append([pair_EF_E, pair_EF_F])
                    # remove the paired edges and faces
                    CR.remove(pair_EF_E)
                    ST.remove(pair_EF_F)
                else:
                    crit_cell.append(CR.popleft())
            elif k == 3: # get FT pairs
                pair_FT = get_FT_pair(ST, crit_cell, Forman_EF, Forman_FT)
                if pair_FT: # the FT pair exists
                    pair_FT_F = pair_FT[0]
                    pair_FT_T = pair_FT[1]
                    Forman_FT.append([pair_FT_F, pair_FT_T])
                    # remove the paired faces and tetra
                    CR.remove(pair_FT_F)
                    ST.remove(pair_FT_T)
                else:
                    crit_cell.append(CR.popleft())
                    
            elif k == 4:
                crit_cell.append(CR.popleft())
                
    return crit_cell, Forman_VE, Forman_EF, Forman_FT
                
# StructType for Crit_cell        
add_Crit_cell_schema = StructType([
    StructField('Crit_cell',ArrayType(IntegerType()),True)
])

# StructType for VE pairs        
add_VE_schema = StructType([
    StructField("VE_pair_V", IntegerType(),True), 
    StructField('VE_pair_E',ArrayType(IntegerType()),True)
])

# StructType for EF pairs        
add_multi_EF_schema = StructType([
    StructField("EF_pair_E", ArrayType(IntegerType()),True), 
    StructField('EF_pair_F',ArrayType(IntegerType()),True)
])

# StructType for FT pairs        
add_multi_FT_schema = StructType([
    StructField("FT_pair_F", ArrayType(IntegerType()),True), 
    StructField('FT_pair_T',ArrayType(IntegerType()),True)
])

# the whole StructType
get_Forman_schema = StructType([
    StructField("Crit_cell", ArrayType(ArrayType(IntegerType())), True),
    StructField("VE_pair", ArrayType(add_VE_schema), True),
    StructField("EF_pair", ArrayType(add_multi_EF_schema), True),
    StructField("FT_pair", ArrayType(add_multi_FT_schema), True)
])

# convert a function to an udf and determine the return type
# https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.udf.html
get_Forman_udf = udf(get_Forman, get_Forman_schema)

In [12]:
df_Forman = df_LS_EFT.withColumn("LS_Forman", get_Forman_udf(df_LS_EFT.Ver, df_LS_EFT.LS_VT, df_LS_EFT.LS_VF, df_LS_EFT.LS_VE))
df_Forman = df_Forman.select(col("Ver"), col("LS_Forman").alias("Forman"))
df_Forman.printSchema()
print("The number of partitions for df_Forman:", df_Forman.rdd.getNumPartitions())
# df_Forman.show()

root
 |-- Ver: integer (nullable = true)
 |-- Forman: struct (nullable = true)
 |    |-- Crit_cell: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |    |-- VE_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- VE_pair_V: integer (nullable = true)
 |    |    |    |-- VE_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- EF_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- EF_pair_E: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |    |    |-- EF_pair_F: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- FT_pair: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- FT_pair_F: array (nullable = true)
 |    | 

In [ ]:
t0 = time.time()

num_row = df_Forman.count()

t1 = time.time()
t_count_Forman = t1 - t0
print("Time cost:", t_count_Forman)
print("number of rows:", num_row)

In [ ]:
print("****************************")
print("Time cost for preparing the relations:", t_count_Forman_rela)

print("****************************")
print("Time cost for computing the Forman gradient:", t_count_Forman)

print("****************************")
print("Total time cost:", t_count_Forman_rela + t_count_Forman)